In [131]:
import os
from pathlib import Path
from collections import OrderedDict, namedtuple
import time
from datetime import datetime
import codecs
from textwrap import wrap

import numpy as np
import wikipediaapi
from moviepy.editor import *
from gtts import gTTS
from skimage.io import imread, imsave
from skimage import transform
from skimage.util import img_as_ubyte

from image_downloader import master_download
from im_funcs import maxsize_pad

from synthesize import synthesize
from hyperparams import Hyperparams as hp
from data_load import *


# Default Parameters
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
WHITE_GIZEH = (1, 1, 1)
BLACK_GIZEH = (0, 0, 0)

VIDEO_SIZE = (1920, 1080)
IMG_SHAPE = (1080, 1920)
IMG_DISPLAY_DURATION = 4    #duration, in seconds, to display each image

excluded_sections = {'See also', 'References', 'Further reading', 'External links',
                'Formats and track listings', 'Credits and personnel', 'Charts',
                'Certifications', 'Release history'}

stops = {'e.g.', '[sic]', '[...]', 'i.e.'}

In [2]:
wiki = wikipediaapi.Wikipedia('en')
page = wiki.page('Bokmål')
page.title

'Bokmål'

### Workflow:

- Initialize WikiMovie with page
- Create txt file
    - Title,
    - Summary text,
    - Section1 title,
    - Section1 text, 
    - etc ....
- Output all .wav speech files with synthesize function
-  ... currently just name files 0.wav, 1.wav, 2 ....
- Generate clips:
    Possible output:
    - textclip, imageseq with 0.wav (section with text to read)
    - textclip with 1.wav (section without text)

    
Basically, will have to iterate through output samples directory (possibly with just index 'i' for filepath),
and link them back with the text.

If Section has no text, it's just a main header... ex: `{'History': '', 'Ancient times': 'blah blah...'}`

Create textclip for 'History', set audio, and go on to next, e.g. 'Ancient times'

If value at the key is not empty, create textclip as above, AND Image Sequence.

For each item try to make one piece of audio continue through Textclip display and Image sequence if necessary.
Maybe just set the section header durations to be a little long, like 3 secs. The actual paragraph may start in sooner but at least the audio won't get cutoff, and don't have to synthesize the section title audio separately on a different line of the text file.

    
|   audio ---------| ------------> |
|----------------:|:--------------|
| /Section title + |         Text/ |
| TextClip         | ImageSequence |


- Pass the dict into text file as `script[i]['title'] + '. ' + script[i]['text']`
  
  (notice the space after the period)
  
- Also process the script dictionary for creating 

In [175]:
class WikiMovie():
    """
    Make movies in standard format (.mp4) from wikipedia pages.
    Initialize with 'page' object from wikipedia Python module.
    Primary user function is make_movie().
    """
    def __init__(self, page, narrator='gtts'):
        self.page = page
        self.title = self.page._attributes['title']
        self.narrator = narrator
        self.script = [{'title': page.title, 
                        'level': 0, 
                        'text': self.clean_text(page.summary)}]
        self.cliplist = []
        
        # self.p = Path(__file__).resolve().parents[1]
        self.p = Path(os.path.abspath('')).resolve() ## For jupyter notebook
        self._imgidx = 0
        self.cutoff = None
        self.longest= 0
        

    def _create_paths(self):
        # Image directories
        self.parent_images = self.p / 'images'
        self.imgdir = self.p / 'images' / self.title
        self.resizedir = self.imgdir / 'resize'
        # gTTS audio directories
        self.parent_audio = self.p / 'audio'
        self.auddir = self.p / 'audio' / self.title
        # dc_tts directory
        self.dctts_dir = self.p / 'dc_tts'
        self.dctts_in = self.dctts_dir / 'text_input'
        self.dctts_out = self.dctts_dir / 'samples' / self.title
        # URL lists text files directory
        self.url_dir = self.p / 'url_files'
        # Video directory (all article videos stored in folder, files named by title)
        self.viddir = self.p / 'videos'
        # Video save path
        self.vidpath = self.viddir / (self.title + ".mp4")

        print('creating paths...')
        for d in [self.parent_images, self.imgdir, self.resizedir,\
                self.parent_audio, self.auddir, self.dctts_dir,\
                self.dctts_in, self.dctts_out, self.url_dir, self.viddir]:
            if not d.exists():
                d.mkdir()
                print(d, "directory created")
            elif d.exists():
                print(d, "exists")

    def _resize_images(self):
        self._imgpaths = []
        contents = self.imgdir.glob('*')
        fnames =  [x for x in contents if x.is_file() and x.parts[-1][0] != '.']
        self.fixed_durations = [IMG_DISPLAY_DURATION for _ in fnames]
        
        n_imgs = len(fnames)
        for i, fname in enumerate(fnames):
            sys.stdout.write(f"Resizing Images [{'#' * (i+1) + ' ' * (n_imgs-i-1)}]   \r")
            sys.stdout.flush()
            
            path = str(self.imgdir / fname)
            print(path)
            save_path = str(self.resizedir / fname)
            print(self.resizedir)
            print(save_path)
            try:
                maxsize_pad(path, save_path)
            except Exception:
                continue
            self._imgpaths.append(save_path)


    def make_narration(self, string, mp3path):
        if self.narrator == 'dc_tts':
            synthesize()
        else:
                             
        tts = gTTS(string)
        tts.save(mp3path)
        return AudioFileClip(mp3path)


    def _add_ImageSequence(self, audioclip):
        """add image with soundtrack audioclip of narrated text"""
        ### Cycle through images so it doesn't always start at the first one
        tmp_imgpaths = self._imgpaths[self._imgidx:] + self._imgpaths[:self._imgidx]
        self._imgidx += 1
        image_sequence = ImageSequenceClip(sequence=tmp_imgpaths,
                                durations=self.fixed_durations, load_images=True).\
                            set_position(('center', 400)).\
                            fx(vfx.loop, duration=audioclip.duration).\
                            set_audio(audioclip)
        self.cliplist.append(image_sequence)


    def _add_subsection(self, section, level):
        """
        Add textclip of section titles.
        If it's just a main header followed by subsections, NO image sequence.
        If section contains text, create narratation and image sequence.
        """

        mp3path_header = os.path.join(self.auddir, section.title + '_header.mp3')
        ac_header = self._make_narration(section.title, mp3path_header)
        fontsize = 130 - (30 * level) # higher level means deeper 'indentation'
        tc_header = TextClip(section.title, color='white', fontsize=fontsize, 
                    size=VIDEO_SIZE, method='caption').\
                    set_audio(ac_header).set_duration(ac_header.duration)
        self.cliplist.append(tc_header)
        ## if there is an actual paragraph in the section, create an image sequence for it
        if section.text:
            mp3path_text = os.path.join(self.auddir, section.title + '_text.mp3')
            ac_text = self._make_narration(section.text[:self.cutoff], mp3path_text)
            self._add_ImageSequence(ac_text)

        print(section.title, "complete")
                             
    def clean_text(self, text):
        """
        Remove stop words
        """
        for x in stops:
            if x in text:
                print(x, "in text")
            text = text.replace(x, '')
        return text

    def flush_sections(self, sections, level=0):
        """
        Get text from all levels (sections, subsections) in page order and generate narrations
        """
        for s in sections:
            if s.title in excluded_sections:
                print('exluding')
                continue
            else:
                self.script.append({'title':s.title, 
                                    'level': level+1, 
                                    'text': self.clean_text(s.text)})
                # self._add_subsection(s, level) # clip creation
                # recursion to next level. Once lowest level is reached, next main section will be accessed
                self.flush_sections(s.sections, level+1)
        
    def output_text(self):
        """
        Process page text for TTS creation. Write to file. 
        """
        self.sent_path = self.dctts_in / f"{self.title}.txt"
        hp.test_data = self.sent_path
        
        with self.sent_path.open('w') as sf:
            sf.write(f"Script for Wikipedia article {self.title}\n")
            
            # Section by section
            for d in self.script[:1]:
                # d = {'title': section title, 'level': level, 'text': section text}

                # Full length sentences, possibly greater than 
                full_sents = [d['title']] + d['text'].split('.')
                             
                # break section down into max 180 character chunks  
                sents = []
                for sent in (full_sents):
                    if len(sent) > hp.max_N:
                        split_on = sent[:hp.max_N].rfind(" ")
                        tmp_split = [sent[:split_on], sent[split_on:]]
                        while len(tmp_split[-1]) > hp.max_N:
                            last = tmp_split.pop()
                            split_on = last[:hp.max_N].rfind(" ")
                            tmp_split += [last[:split_on], last[split_on:]]
                        sents.extend(tmp_split)     
                    else:
                        sents.append(sent)         
                     
                for i, sent in enumerate(sents):     
                    sf.write(f"{d['title']}:{i} {sent}\n")
    
    def create_clips():
        pass
                         
    def make_movie(self, cutoff=None):
        """
        Args:
            cutoff (int): Limit the length of the script. Used like script[:cutoff]
        Returns:
            None
        """
        print("Video Title: ", self.title)
        self.cutoff = cutoff
        self._create_paths()
        
        self.output_text()
        self.make_narration()
        # Download and resize images
        master_download(main_keyword=self.title, 
                        url_dir=self.url_dir, img_dir=self.imgdir, num_requested=20)
        self._resize_images()
        print('\n') 
                             
        # Create Video Clips
        print("Creating clips. . .")
#         self._flush_page()

                             
        thanks = TextClip("Thanks for watching \n and listening",
                            color='white', fontsize=72, size=VIDEO_SIZE, method='caption').\
                            set_duration(2)

        subscribe = TextClip("Please Subscribe!",
                                color='white', fontsize=72, size=VIDEO_SIZE, method='caption').\
                                set_duration(2)

        self.video = concatenate_videoclips(self.cliplist + [thanks, subscribe],
                                            method='compose').\
                                            on_color(color=BLACK, col_opacity=1)
        # Encode Video
        start = datetime.now()
        self.video.write_videofile(str(self.vidpath) , fps=1, codec='mpeg4', 
                                   audio_codec="aac", preset='ultrafast')
        dur = datetime.now() - start
        print("Video Encoding completed in time: ", dur)

        # self.audio_clip.close()
        # title_text.close()
        thanks.close()
        subscribe.close()
        self.video.close()
        

In [176]:
WMM = WikiMovie(page)

[...] in text
e.g. in text
[sic] in text


In [177]:
WMM._create_paths()

creating paths...
/Users/jared/video-creater/images exists
/Users/jared/video-creater/images/Bokmål exists
/Users/jared/video-creater/images/Bokmål/resize exists
/Users/jared/video-creater/audio exists
/Users/jared/video-creater/audio/Bokmål exists
/Users/jared/video-creater/dc_tts exists
/Users/jared/video-creater/dc_tts/text_input exists
/Users/jared/video-creater/dc_tts/samples/Bokmål exists
/Users/jared/video-creater/url_files exists
/Users/jared/video-creater/videos exists


In [179]:
WMM.flush_sections(page.sections)
WMM.output_text()
hp.test_data = str(WMM.sent_path)
hp.sampledir = str(WMM.dctts_out)

exluding


In [184]:
lines = codecs.open(WMM.sent_path, 'r', 'utf-8').readlines()[1:]

In [185]:
lines

['Bokmål:0 Bokmål\n',
 'Bokmål:1 Bokmål (UK: , US: ; literally "book tongue") is an official written standard for the Norwegian language, alongside Nynorsk\n',
 'Bokmål:2  Bokmål is the preferred written standard of Norwegian for 85% to 90% of the population in Norway\n',
 'Bokmål:3  Unlike, for instance, the Italian language, there is no nationwide standard or agreement on the pronunciation of Bokmål\n',
 'Bokmål:4 \n',
 'Bokmål is regulated by the governmental Norwegian Language Council\n',
 'Bokmål:5  A more conservative orthographic standard, commonly known as Riksmål, is regulated by the non-governmental Norwegian Academy for Language and Literature\n',
 'Bokmål:6  The written standard is a Norwegianised variety of the Danish language\n',
 'Bokmål:7 \n',
 'The first Bokmål orthography was officially adopted in 1907 under the name Riksmål after being under development since 1879\n',
 'Bokmål:8  The architects behind the reform were Marius Nygaard and Jacob Jonathan Aars\n',
 'Bokmå

In [186]:
sents = [text_normalize(line.split(" ", 1)[-1]).strip() + "E" for line in lines] # text normalization, E: EOS
sents = [s for s in sents if s != 'E']

In [187]:
sents

['bokmalE',
 'bokmal uk us literally book tongue is an official written standard for the norwegian language alongside nynorskE',
 'bokmal is the preferred written standard of norwegian for to of the population in norwayE',
 'unlike for instance the italian language there is no nationwide standard or agreement on the pronunciation of bokmalE',
 'is regulated by the governmental norwegian language councilE',
 'a more conservative orthographic standard commonly known as riksmal is regulated by the non governmental norwegian academy for language and literatureE',
 'the written standard is a norwegianised variety of the danish languageE',
 'first bokmal orthography was officially adopted in under the name riksmal after being under development sinceE',
 'the architects behind the reform were marius nygaard and jacob jonathan aarsE',
 'it was an adaptation of written danish which was commonly used since the past union with denmark to the dano norwegian koine spoken by the norwegian urban elit

In [195]:
char2idx, idx2char = load_vocab()
texts = np.zeros((len(sents), hp.max_N), np.int32)
for i, sent in enumerate(sents):
    texts[i, :len(sent)] = [char2idx[char] for char in sent]

print(f"{texts.shape[0]} samples, {texts.shape[1]} characters (max) each")

26 samples, 180 characters (max) each


In [208]:
print("A sample of the text encoding")
print(texts[:5, :20])

A sample of the text encoding
[[ 4 17 13 15  3 14  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4 17 13 15  3 14  2 23 13  2 23 21  2 14 11 22  7 20  3 14]
 [ 4 17 13 15  3 14  2 11 21  2 22 10  7  2 18 20  7  8  7 20]
 [23 16 14 11 13  7  2  8 17 20  2 11 16 21 22  3 16  5  7  2]
 [11 21  2 20  7  9 23 14  3 22  7  6  2  4 27  2 22 10  7  2]]


In [153]:
synthesize()




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Use keras.layers.dropout instead.

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
Graph loaded


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from LJ_logdir/LJ01-1/model_gs_724k
Text2Mel Restored!
INFO:tensorflow:Restoring parameters from LJ_logdir/LJ01-2/model_gs_718k


  0%|          | 0/210 [00:00<?, ?it/s]

SSRN Restored!


100%|██████████| 210/210 [19:00<00:00,  5.43s/it]


Working on file 1
Working on file 2
Working on file 3
Working on file 4
Working on file 5
Working on file 6
Working on file 7
Working on file 8
Working on file 9
Working on file 10
Working on file 11
Working on file 12
Working on file 13
Working on file 14
Working on file 15
Working on file 16
Working on file 17
Working on file 18
Working on file 19
Working on file 20
Working on file 21
Working on file 22
Working on file 23
Working on file 24
Working on file 25
Working on file 26
Working on file 27
Working on file 28
Working on file 29
Working on file 30
Working on file 31
